In [1]:
import google.generativeai as genai
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import json

# Configure the Gemini API
genai.configure(api_key='GOOGLE_API_KEY')

# Initialize the Gemini model
model = genai.GenerativeModel('gemini-pro')

# Finance-focused system prompt
FINANCE_SYSTEM_PROMPT = """
You are an AI assistant specifically designed to provide information and answer questions about finance and economics. 
Your knowledge encompasses a wide range of financial topics, including personal finance, investing, banking, corporate finance, economic theory, global markets, cryptocurrency, financial regulations, tax planning, retirement planning, insurance, financial analysis, risk management, mergers and acquisitions, and venture capital.

You should only respond to queries related to these financial topics. If a user asks a question that is not related to finance or economics, politely inform them that you are specialized in financial matters and cannot assist with their current query. 
Always strive to provide accurate, up-to-date information and clarify that for specific financial advice, users should consult with a qualified financial professional.

Maintain a professional and educational tone, explaining complex terms when necessary. 
Your goal is to help users understand financial concepts and make informed decisions, not to provide specific investment advice or predictions about market performance.

If you're unsure about any information or if a question requires very recent market data, acknowledge the limitations of your knowledge and suggest where the user might find the most current information.

Remember, you are not authorized to make financial transactions, access personal financial information, or provide personalized financial advice. 
Always encourage users to do their own research and consult with licensed financial advisors for personalized guidance.
"""

class AIFinancialAdvisor:
    def __init__(self):
        self.portfolio = {}
        self.transaction_history = []
        self.expenses = []
        self.income = []
        self.debts = []
        self.savings_goals = []
        self.user_model = None
        self.market_model = None
        self.scaler = StandardScaler()

    def get_real_time_data(self, symbol, period="1d", interval="1m"):
        stock = yf.Ticker(symbol)
        data = stock.history(period=period, interval=interval)
        return data

    def get_stock_info(self, symbol):
        stock = yf.Ticker(symbol)
        info = stock.info
        current_price = info['regularMarketPrice']
        previous_close = info['previousClose']
        change = current_price - previous_close
        change_percent = (change / previous_close) * 100
        
        return f"""
        Stock: {symbol.upper()}
        Current Price: ${current_price:.2f}
        Previous Close: ${previous_close:.2f}
        Change: ${change:.2f} ({change_percent:.2f}%)
        52 Week High: ${info['fiftyTwoWeekHigh']:.2f}
        52 Week Low: ${info['fiftyTwoWeekLow']:.2f}
        Volume: {info['volume']:,}
        Market Cap: ${info['marketCap']:,}
        """

    def buy_shares(self, symbol, quantity):
        data = self.get_real_time_data(symbol)
        price = data['Close'].iloc[-1]
        total_cost = price * quantity
        if symbol in self.portfolio:
            self.portfolio[symbol] += quantity
        else:
            self.portfolio[symbol] = quantity
        self.transaction_history.append({
            'date': datetime.now(),
            'action': 'buy',
            'symbol': symbol,
            'quantity': quantity,
            'price': price
        })
        return f"Bought {quantity} shares of {symbol} at ${price:.2f} each. Total cost: ${total_cost:.2f}"

    def sell_shares(self, symbol, quantity):
        if symbol not in self.portfolio or self.portfolio[symbol] < quantity:
            return f"Error: Not enough shares of {symbol} to sell."
        data = self.get_real_time_data(symbol)
        price = data['Close'].iloc[-1]
        total_value = price * quantity
        self.portfolio[symbol] -= quantity
        if self.portfolio[symbol] == 0:
            del self.portfolio[symbol]
        self.transaction_history.append({
            'date': datetime.now(),
            'action': 'sell',
            'symbol': symbol,
            'quantity': quantity,
            'price': price
        })
        return f"Sold {quantity} shares of {symbol} at ${price:.2f} each. Total value: ${total_value:.2f}"

    def add_expense(self, amount, category):
        self.expenses.append({'date': datetime.now(), 'amount': amount, 'category': category})
        return f"Added expense of ${amount:.2f} in category {category}"

    def add_income(self, amount, source):
        self.income.append({'date': datetime.now(), 'amount': amount, 'source': source})
        return f"Added income of ${amount:.2f} from {source}"

    def add_debt(self, amount, creditor, interest_rate):
        self.debts.append({'amount': amount, 'creditor': creditor, 'interest_rate': interest_rate})
        return f"Added debt of ${amount:.2f} to {creditor} at {interest_rate}% interest rate"

    def add_savings_goal(self, amount, name, target_date):
        self.savings_goals.append({'amount': amount, 'name': name, 'target_date': target_date})
        return f"Added savings goal of ${amount:.2f} for {name} by {target_date}"

    def get_current_dti(self):
        total_monthly_debt = sum(debt['amount'] for debt in self.debts) / 12
        total_monthly_income = sum(income['amount'] for income in self.income) / 12
        if total_monthly_income == 0:
            return "Cannot calculate DTI: No income recorded"
        dti = (total_monthly_debt / total_monthly_income) * 100
        return f"Your current Debt-to-Income (DTI) ratio is {dti:.2f}%"

    def estimate_debt_payoff_time(self):
        total_debt = sum(debt['amount'] for debt in self.debts)
        avg_interest_rate = sum(debt['interest_rate'] for debt in self.debts) / len(self.debts)
        monthly_payment = sum(income['amount'] for income in self.income) / 12 * 0.2  # Assuming 20% of income goes to debt
        months = 0
        remaining_debt = total_debt
        while remaining_debt > 0:
            interest = remaining_debt * (avg_interest_rate / 100 / 12)
            remaining_debt += interest
            remaining_debt -= monthly_payment
            months += 1
        years = months // 12
        remaining_months = months % 12
        return f"Estimated time to clear all debt: {years} years and {remaining_months} months"

    def analyze_user_behavior(self):
        if len(self.transaction_history) < 10:
            return "Not enough transaction history to analyze user behavior."

        df = pd.DataFrame(self.transaction_history)
        df['day_of_week'] = df['date'].dt.dayofweek
        df['hour'] = df['date'].dt.hour

        X = df[['day_of_week', 'hour', 'price']]
        y = df['action'].map({'buy': 1, 'sell': 0})

        X = self.scaler.fit_transform(X)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        self.user_model = RandomForestClassifier(n_estimators=100, random_state=42)
        self.user_model.fit(X_train, y_train)

        accuracy = self.user_model.score(X_test, y_test)
        return f"User behavior model trained with accuracy: {accuracy:.2f}"

    def get_recommendation(self, symbol):
        if self.user_model is None:
            return "Not enough data to make a personalized recommendation."

        data = self.get_real_time_data(symbol)
        current_price = data['Close'].iloc[-1]
        current_time = datetime.now()

        features = np.array([[current_time.weekday(), current_time.hour, current_price]])
        scaled_features = self.scaler.transform(features)

        prediction = self.user_model.predict(scaled_features)
        probabilities = self.user_model.predict_proba(scaled_features)

        if prediction[0] == 1:
            action = "buy"
        else:
            action = "sell"

        confidence = probabilities[0][prediction[0]]

        return f"Based on your past behavior, the recommendation is to {action} {symbol} with {confidence:.2f} confidence."

    def predict_next_day_price(self, symbol):

        # Load LSTM model from the other file
        from forecast import get_stock_data_and_forecast 

        # Get past prices and forecast from the LSTM model
        past_prices, future_forecast = get_stock_data_and_forecast(symbol)

        # Extract the predicted price for the next day
        predicted_price = future_forecast[0] 
        
        return f"Predicted next day close price for {symbol} is ${predicted_price:.2f}"
    def check_finances(self):
        """Provides a complete overview of the user's finances."""
        total_assets = 0
        total_liabilities = 0
        net_worth = 0

        # Calculate portfolio value
        for symbol, quantity in self.portfolio.items():
            data = self.get_real_time_data(symbol)
            price = data['Close'].iloc[-1]
            total_assets += quantity * price

        # Calculate total income and expenses
        total_income = sum(income['amount'] for income in self.income)
        total_expenses = sum(expense['amount'] for expense in self.expenses)

        # Calculate total debt
        total_liabilities = sum(debt['amount'] for debt in self.debts)

        # Calculate net worth
        net_worth = total_assets - total_liabilities

        # Calculate DTI
        dti = self.get_current_dti()

        overview = f"""
        Financial Overview:

        Portfolio Value: ${total_assets:.2f}
        Total Income: ${total_income:.2f}
        Total Expenses: ${total_expenses:.2f}
        Total Debt: ${total_liabilities:.2f}
        Net Worth: ${net_worth:.2f}
        {dti}

        Portfolio Holdings:
        """

        # Print portfolio details
        if self.portfolio:
            for symbol, quantity in self.portfolio.items():
                data = self.get_real_time_data(symbol)
                price = data['Close'].iloc[-1]
                value = quantity * price
                overview += f"  - {symbol.upper()}: {quantity} shares, ${value:.2f}\n"
        else:
            overview += "  - No portfolio holdings currently.\n"

        # Print debts
        if self.debts:
            overview += "\nDebts:\n"
            for debt in self.debts:
                overview += f"  - {debt['creditor']}: ${debt['amount']:.2f} ({debt['interest_rate']}% interest rate)\n"
        else:
            overview += "\nNo debts currently.\n"

        return overview

    def process_command(self, command):
        try:
            if command.startswith("buy"):
                _, symbol, quantity = command.split()
                return self.buy_shares(symbol.upper(), int(quantity))
            elif command.startswith("sell"):
                _, symbol, quantity = command.split()
                return self.sell_shares(symbol.upper(), int(quantity))
            elif command.startswith("expense"):
                _, amount, category = command.split()
                return self.add_expense(float(amount), category)
            elif command.startswith("income"):
                _, amount, source = command.split()
                return self.add_income(float(amount), source)
            elif command.startswith("debt"):
                _, amount, creditor, interest_rate = command.split()
                return self.add_debt(float(amount), creditor, float(interest_rate))
            elif command.startswith("savings_goal"):
                _, amount, name, target_date = command.split()
                return self.add_savings_goal(float(amount), name, target_date)
            elif command == "dti":
                return self.get_current_dti()
            elif command == "payoff_time":
                return self.estimate_debt_payoff_time()
            elif command.startswith("recommend"):
                _, symbol = command.split()
            elif command.startswith("predict"):
                _, symbol = command.split()
                return self.predict_next_day_price(symbol.upper())
            elif command.startswith("info"):
                _, symbol = command.split()
                return self.get_stock_info(symbol.upper())
            elif command == "check":
                return self.check_finances() # Added check command
            else:
                return "Unknown command. Please check your input."
        except Exception as e:
            return f"Error processing command: {str(e)}"
        
    def get_ai_response(self, user_input):
        response = model.predict(prompt=FINANCE_SYSTEM_PROMPT + user_input, max_output_tokens=150)
        return response.content # Access the content directly from the 'response' object

    def run(self):
        print("Welcome to the AI Financial Advisor. Type 'help' for a list of commands.")
        while True:
            user_input = input("> ")
            if user_input.lower() == 'exit':
                break
            elif user_input.lower() == 'help':
                print("""
                Available Commands:
                - buy [symbol] [quantity]: Buy shares of a stock.
                - sell [symbol] [quantity]: Sell shares of a stock.
                - expense [amount] [category]: Add an expense.
                - income [amount] [source]: Add income.
                - debt [amount] [creditor] [interest_rate]: Add debt.
                - savings_goal [amount] [name] [target_date]: Add a savings goal.
                - dti: Calculate Debt-to-Income ratio.
                - payoff_time: Estimate debt payoff time.
                - recommend [symbol]: Get a stock recommendation based on behavior.
                - predict [symbol]: Predict next day's stock price.
                - info [symbol]: Get real-time stock information.
                - check: Check your financial overview.
                - exit: Exit the assistant.
                """)
            else:
                if user_input.startswith("ai:"):
                    ai_query = user_input[3:].strip()
                    print(self.get_ai_response(ai_query))
                else:
                    print(self.process_command(user_input))

# Run the assistant
advisor = AIFinancialAdvisor()
advisor.run()

Welcome to the AI Financial Advisor. Type 'help' for a list of commands.


>  EXIT


In [ ]:
import yfinance as yf

# Class to simulate account and stock transactions
class StockTradingBot:
    def __init__(self, balance=0):
        self.balance = balance
        self.portfolio = {}  # Store stocks and number of shares
        self.transaction_history = []  # To store buy/sell history

    def deposit_funds(self, amount):
        self.balance += amount
        return f"${amount} has been deposited. Current balance: ${self.balance}"

    def check_balance(self):
        return f"Current balance: ${self.balance}"

    def buy_shares(self, ticker, num_shares):
        stock = yf.Ticker(ticker)
        current_price = stock.history(period="1d")['Close'][0]  # Get latest closing price
        total_cost = current_price * num_shares

        if self.balance >= total_cost:
            self.balance -= total_cost
            if ticker in self.portfolio:
                self.portfolio[ticker] += num_shares
            else:
                self.portfolio[ticker] = num_shares

            self.transaction_history.append(f"Bought {num_shares} shares of {ticker} at ${current_price} each.")
            return f"Bought {num_shares} shares of {ticker} for ${total_cost}. Remaining balance: ${self.balance}"
        else:
            return f"Not enough funds to buy {num_shares} shares of {ticker}. You need ${total_cost}, but only have ${self.balance}."

    def sell_shares(self, ticker, num_shares):
        stock = yf.Ticker(ticker)
        current_price = stock.history(period="1d")['Close'][0]  # Get latest closing price

        if ticker in self.portfolio and self.portfolio[ticker] >= num_shares:
            total_sale = current_price * num_shares
            self.balance += total_sale
            self.portfolio[ticker] -= num_shares
            if self.portfolio[ticker] == 0:
                del self.portfolio[ticker]

            self.transaction_history.append(f"Sold {num_shares} shares of {ticker} at ${current_price} each.")
            return f"Sold {num_shares} shares of {ticker} for ${total_sale}. New balance: ${self.balance}"
        else:
            return f"Not enough shares of {ticker} to sell. You own {self.portfolio.get(ticker, 0)} shares."

    def view_portfolio(self):
        portfolio_summary = "Current Portfolio:\n"
        for ticker, shares in self.portfolio.items():
            portfolio_summary += f"{ticker}: {shares} shares\n"
        portfolio_summary += f"Current balance: ${self.balance}"
        return portfolio_summary

    def view_transactions(self):
        transaction_history = "Transaction History:\n"
        for transaction in self.transaction_history:
            transaction_history += transaction + "\n"
        return transaction_history

# Simple chatbot-like interaction
def chatbot():
    bot = StockTradingBot()
    print("Welcome to Fintastic Talk+ ! How can I assist you today?")
    
    while True:
        user_input = input("You: ").lower()

        if 'deposit' in user_input:
            amount = float(input("How much would you like to deposit? "))
            print(bot.deposit_funds(amount))

        elif 'buy' in user_input:
            ticker = input("Which stock (ticker) would you like to buy? ").upper()
            num_shares = int(input(f"How many shares of {ticker} would you like to buy? "))
            print(bot.buy_shares(ticker, num_shares))

        elif 'sell' in user_input:
            ticker = input("Which stock (ticker) would you like to sell? ").upper()
            num_shares = int(input(f"How many shares of {ticker} would you like to sell? "))
            print(bot.sell_shares(ticker, num_shares))

        elif 'portfolio' in user_input:
            print(bot.view_portfolio())

        elif 'transactions' in user_input:
            print(bot.view_transactions())

        elif 'balance' in user_input:
            print(bot.check_balance())

        elif 'exit' in user_input:
            print("Goodbye! Have a great day investing!")
            break

        else:
            print("I didn't understand that. Please say 'deposit', 'buy', 'sell', 'portfolio', 'transactions', 'balance', or 'exit'.")

# Run the chatbot
chatbot()
